# <center> **Распознавание речи в текст на русском языке**
# <center> **(задание от ЭКСМО)**

Импорт библиотек 

In [28]:
import pandas as pd # чтение данных
from numpy.random import randint # случайное число
import json # для сохраения текстовых файлов

import warnings as w # чтобы предупреждения не мешали
w.filterwarnings('ignore')

## Данные

In [29]:
data = pd.read_json('data/manifest.jsonl', lines=True) # данные нужно прочитать 

data

,id,audio_filepath,text,duration
0,e632f7d39c15e7edfc665b91e6f2071f,files/e632f7d39c15e7edfc665b91e6f2071f.wav,афина воспроизведи музыку вперемешку,4.900000
1,5db5df8bb9e3b6660b2a04b34d4a355d,files/5db5df8bb9e3b6660b2a04b34d4a355d.wav,найти сериал григорий р,3.652000
2,2c471aedc6979109f28cd53c58f8c4fb,files/2c471aedc6979109f28cd53c58f8c4fb.wav,прямой эфир апл манчестер юнайтед тоттенхэм,4.341750
3,756a137ee9debde4a008adc4a4121dc7,files/756a137ee9debde4a008adc4a4121dc7.wav,ильвиром ивановичем ворончихиным,3.900000
4,1ee3b00170123a6723a40e129b2f6bce,files/1ee3b00170123a6723a40e129b2f6bce.wav,можешь показать киношку исходный код,3.320000
...,...,...,...,...
9989,73bb9272fe3724eb6212e33186f63152,files/73bb9272fe3724eb6212e33186f63152.wav,рудню,2.260000
9990,c872c6f50709a2696b0927725dc86ded,files/c872c6f50709a2696b0927725dc86ded.wav,найти мульт щенячий патруль,3.020000
9991,03b92263995462b2e09655a8e29d4d0b,files/03b92263995462b2e09655a8e29d4d0b.wav,стол из эпоксидной смолы,3.598688
9992,09ac1f9718c6da18380b4a01d92d2561,files/09ac1f9718c6da18380b4a01d92d2561.wav,шесть четыреста семьдесят шесть четыреста двад...,11.795312


Моя идея состоит в том, чтобы случайным образом выбирать аудиофайл для дальнейшего чтения.

In [30]:
i = randint(0, data.shape[0]) # случайное число в диапазоне от 0 до последнего индекса таблицы `data`

audio_id = data['id'].iloc[i] # название файла 
audio_path = 'data/' + data['audio_filepath'].iloc[i] # путь к файлу
audio_text = data['text'].iloc[i] # реальный текст файла (для проверки качества распознавания)


# создадим функцию для оценки качества распознавания и сохранения результата в текстовый формат
def compare_and_dump(result, real, audio_id): 
    """ 
    Функция выводит на экран прочитанное сообщение (`result`), 
    затем выводит реальный текст (`real`), 
    и сравнивает, сходится ли первое co вторым. 
    
    В конечном счете результат сохраняется в файл `.txt` в отдельную папку,
    и выводится путь к этому файлу.
    
    """
    # приводим оба текста к единому регистру
    result_str = str(result).lower() 
    real_str = str(real).lower()
    
    # булева переменная, идентично ли одно другому
    similarity = result_str == real_str 
    
    # вывод на экран сравнения результата и реального текста
    print(f'Recognized is "{result_str}",',  
          f'\nwhile real is "{real_str}".',  
          f'\nSimilarity is {similarity}')
    
    # путь к созданному файлу с результатом
    result_path = f'texts/{audio_id}_text.txt'
    
    # сохранение распознанного текста
    with open(result_path, 'w') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

    # вывод того, куда был сохранен текст
    print('\nResult has been saved into '+result_path)

## Speech Recognition (google)

Для начала попробуем самый простой вариант - распознавание речи с помощью google. Реализуем его с помощью библиотеки SpeechRecognition.

In [31]:
import speech_recognition as sr # импорт SpeechRecognition

In [32]:
recog = sr.Recognizer() # инициализация распознавателя

# записываем аудио на распознаватель
with sr.AudioFile(audio_path) as audio:
    rec = recog.record(audio)

# распознаем аудио с помощью гугла, указываем русский язык
result = recog.recognize_google(audio_data=rec, language='ru-RU') 

# сравниваем и сохраняем результат
audio_id_google = audio_id + '_google'
compare_and_dump(result, audio_text, audio_id_google)

Recognized is "найди взрывная девушка", 
while real is "найди взрывная девушка". 
Similarity is True

Result has been saved into texts/944a1912c6c9eda5236d4221b93d1bf1_google_text.txt


> В большинстве случаев google справляется неплохо с распознванием русской речи. 

## VOSK

Попробуем теперь вариант посложнее - библиотека VOSK. Для нее я предварительно скачал русскоязычную модель. 

In [33]:
from vosk import Model, KaldiRecognizer # импорт VOSK
from pydub import AudioSegment # из `pydub` берем AudioSegment для предобработки аудио

In [34]:
model = Model("vosk_model") # инициализируем модель
recog = KaldiRecognizer(model, 16000) # выбираем распознаватель
recog.SetWords(True) # разрешить слова

# делаем предобработку аудио (wav)
wav = (AudioSegment.from_wav(audio_path)
       .set_channels(1)
       .set_frame_rate(16000))

# преобразуем вывод в json
recog.AcceptWaveform(wav.raw_data)
result = recog.Result()
text = json.loads(result)["text"] # выбираем из результата только текст

# сравниваем и сохраняем результат
audio_id_vosk = audio_id + '_vosk'
compare_and_dump(text, audio_text, audio_id_vosk)

Recognized is "найди", 
while real is "найди взрывная девушка". 
Similarity is False

Result has been saved into texts/944a1912c6c9eda5236d4221b93d1bf1_vosk_text.txt


> Данная модель справляется намного хуже, чем google. К тому же и работает в среднем от 2-2.5 минут. Так что в качестве конечного результата используем google. 

> *Я использовал всего 2 модели, потому что многие из них требуют API, но его часто получить невозможно, так как у меня недостаточно прав для его получения, либо многие модели просто не устанавливаются.*

## Конечная функция

In [37]:
def google_recog(audio_file: str, dir_save_to: str): 
    """
    Распознавание русской речи с помощью модели Google. 
    
    Аргументы:
        `audio_file` (str) - путь к файлу в строковом формате (формат файла должен состоять из 3 букв). \n
        `dir_save_to` (str) - папка в которую нужно сохранить результат распознавание.
        
    Функция ничего не возвращает, только выводит результат на экран 
    и сохраняет его в формате `.txt` в указанную папку.
    """
    
    recog = sr.Recognizer() # инициализация распознавателя

    # записываем аудио на распознаватель
    with sr.AudioFile(audio_file) as audio:
        rec = recog.record(audio)

    # распознаем аудио с помощью гугла, указываем русский язык
    result = recog.recognize_google(audio_data=rec, language='ru-RU') 

    print(result)
    
    # сравниваем и сохраняем результат
    filename = audio_file.split('/')[-1][:-4] # выделяем имя файла 
    result_path = f'{dir_save_to}/{filename}_google_text.txt' # путь к файлу
    
    with open(result_path, 'w') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

In [40]:
google_recog(audio_path, 'texts')

Найди взрывная девушка
